In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [46]:
df = pd.read_csv('..\data\dataset_algo.csv.zip', index_col=0)

In [47]:
df.columns

Index(['track_name', 'popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'mode', 'speechiness',
       'tempo', 'valence', 'A Capella', 'Alternative', 'Anime', 'Blues',
       'Children's Music', 'Classical', 'Comedy', 'Country', 'Dance',
       'Electronic', 'Folk', 'Hip-Hop', 'Indie', 'Jazz', 'Movie', 'Opera',
       'Pop', 'R&B', 'Rap', 'Reggae', 'Reggaeton', 'Rock', 'Ska', 'Soul',
       'Soundtrack', 'World', 'A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F',
       'F#', 'G', 'G#', 'popularity_score'],
      dtype='object')

In [48]:
cols = ['acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'valence', 'A Capella',
       'Alternative', 'Anime', 'Blues', "Children's Music",
       'Classical', 'Comedy', 'Country', 'Dance', 'Electronic', 'Folk',
       'Hip-Hop', 'Indie', 'Jazz', 'Movie', 'Opera', 'Pop', 'R&B', 'Rap',
       'Reggae', 'Reggaeton', 'Rock', 'Ska', 'Soul', 'Soundtrack', 'World']

X = df[cols]
y = df['popularity_score']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75)

# Create and fit a scaler model
scaler = StandardScaler().fit(X_train)

# Your scaler model can now transform your data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
# Logistic Regression
model = LogisticRegression(max_iter=10).fit(X_train_scaled,y_train)

print("accuracy score on logistic regression train set:",round(model.score(X_train_scaled, y_train), 5))
print("accuracy score on logistic regression test set:",round(model.score(X_test_scaled, y_test), 5))
print ("")

accuracy score on logistic regression train set: 0.67465
accuracy score on logistic regression test set: 0.67391



C:\Users\Berenger\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
